PASO 1: Descargar el Conjunto de Datos desde Google Drive


In [1]:
# Importamos las bibliotecas necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from imblearn.over_sampling import SMOTE

# 1. Instalamos librerías necesarias para trabajar con Google Sheets
!pip install --upgrade gspread gspread-dataframe

# 2. Importamos librerías de Google
import gspread
from gspread_dataframe import get_as_dataframe
from google.colab import auth
from oauth2client.client import GoogleCredentials

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: gspread
    Found existing installation: gspread 6.2.0
    Uninstalling gspread-6.2.0:
      Successfully uninstalled gspread-6.2.0


In [2]:
import gspread
from google.colab import auth
from google.auth import default

# Autenticarse en Google
auth.authenticate_user()

# Obtener credenciales correctas
creds, _ = default()

# Autorizar gspread con las credenciales modernas
gc = gspread.authorize(creds)


In [3]:

spreadsheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1HF_n1Vw8UCzIW91rTJiBhXjEqRLIH1rFbHVXdh6SOYY/edit?usp=sharing')
worksheet = spreadsheet.get_worksheet(1)

# Cargar en dataframe
df = get_as_dataframe(worksheet)

# Mostrar
df.head()


,ANO_NAC,SEXO,IMP_EMO,IMP_APR,EDO_PROC,REG_PROC,MOD_BAC,PROM_BAC,DES_DURO,ESCO_PAD,SER_INTE,BIEN_PC,SER_AUTO,ICNE,IPMA,IPAN,IELE*,ICLE*,class
0,1996.0,1.0,2.0,2.0,29.0,3.0,1.0,7.0,3.0,6.0,2.0,1.0,2.0,964.0,1060.0,1060.0,916.0,820.0,NO SUFICIENTE
1,1996.0,2.0,1.0,2.0,29.0,3.0,1.0,9.3,5.0,2.0,2.0,1.0,1.0,886.0,916.0,892.0,892.0,844.0,NO SUFICIENTE
2,1996.0,1.0,2.0,2.0,29.0,3.0,2.0,8.6,4.0,2.0,1.0,3.0,2.0,1084.0,1084.0,1108.0,1060.0,1084.0,NIVEL SATISFACTORIO
3,1995.0,1.0,2.0,2.0,29.0,3.0,2.0,9.8,5.0,5.0,2.0,2.0,1.0,1204.0,1276.0,1132.0,1252.0,1156.0,NIVEL SOBRESALIENTE
4,1996.0,1.0,2.0,2.0,29.0,3.0,1.0,7.3,4.0,3.0,2.0,2.0,2.0,1102.0,1108.0,1132.0,1036.0,1132.0,NIVEL SATISFACTORIO


## EDA


In [4]:
print(df.shape)
df.head()
df.info()     # tipos y nulos
df['class'].value_counts()  # ¿hay desbalance?


(813, 19)
<class 'pandas.core.frame.DataFrame'>
Index: 813 entries, 0 to 812
Data columns (total 19 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ANO_NAC   813 non-null    float64
 1   SEXO      813 non-null    float64
 2   IMP_EMO   813 non-null    float64
 3   IMP_APR   813 non-null    float64
 4   EDO_PROC  813 non-null    float64
 5   REG_PROC  813 non-null    float64
 6   MOD_BAC   813 non-null    float64
 7   PROM_BAC  813 non-null    float64
 8   DES_DURO  813 non-null    float64
 9   ESCO_PAD  813 non-null    float64
 10  SER_INTE  813 non-null    float64
 11  BIEN_PC   813 non-null    float64
 12  SER_AUTO  813 non-null    float64
 13  ICNE      813 non-null    float64
 14  IPMA      813 non-null    float64
 15  IPAN      813 non-null    float64
 16  IELE*     813 non-null    float64
 17  ICLE*     813 non-null    float64
 18  class     813 non-null    object 
dtypes: float64(18), object(1)
memory usage: 127.0+ KB


,count
class,
NO SUFICIENTE,390
NIVEL SATISFACTORIO,372
NIVEL SOBRESALIENTE,51


In [5]:
# Conversión de tipos de datos

# Lista de columnas a convertir
cols_to_convert = ['SEXO', 'IMP_EMO', 'IMP_APR', 'EDO_PROC', 'REG_PROC',
                   'MOD_BAC', 'DES_DURO', 'ESCO_PAD', 'SER_INTE', 'BIEN_PC', 'SER_AUTO']

# Asegurar que no hay valores nulos
assert df[cols_to_convert].isna().sum().sum() == 0, "Hay valores nulos en las columnas"

# Convertir a enteros (asumiendo que las columnas tienen valores mapeables a enteros)
# Si las columnas tienen strings, usar un mapeo o codificación previa
for col in cols_to_convert:
    # Ejemplo: si son strings, codificar a enteros (ajustar según los datos reales)
    if df[col].dtype == 'object':
        unique_values = df[col].unique()
        mapping = {val: idx for idx, val in enumerate(unique_values)}
        df[col] = df[col].map(mapping).astype('int32')
    else:
        df[col] = df[col].astype('int32')

# Convertir todas las columnas a category de una vez
df[cols_to_convert] = df[cols_to_convert].astype('category')

# Verificar los tipos de datos
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 813 entries, 0 to 812
Data columns (total 19 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   ANO_NAC   813 non-null    float64 
 1   SEXO      813 non-null    category
 2   IMP_EMO   813 non-null    category
 3   IMP_APR   813 non-null    category
 4   EDO_PROC  813 non-null    category
 5   REG_PROC  813 non-null    category
 6   MOD_BAC   813 non-null    category
 7   PROM_BAC  813 non-null    float64 
 8   DES_DURO  813 non-null    category
 9   ESCO_PAD  813 non-null    category
 10  SER_INTE  813 non-null    category
 11  BIEN_PC   813 non-null    category
 12  SER_AUTO  813 non-null    category
 13  ICNE      813 non-null    float64 
 14  IPMA      813 non-null    float64 
 15  IPAN      813 non-null    float64 
 16  IELE*     813 non-null    float64 
 17  ICLE*     813 non-null    float64 
 18  class     813 non-null    object  
dtypes: category(11), float64(7), object(1)
memory usage: 67

In [6]:
df.head()

,ANO_NAC,SEXO,IMP_EMO,IMP_APR,EDO_PROC,REG_PROC,MOD_BAC,PROM_BAC,DES_DURO,ESCO_PAD,SER_INTE,BIEN_PC,SER_AUTO,ICNE,IPMA,IPAN,IELE*,ICLE*,class
0,1996.0,1,2,2,29,3,1,7.0,3,6,2,1,2,964.0,1060.0,1060.0,916.0,820.0,NO SUFICIENTE
1,1996.0,2,1,2,29,3,1,9.3,5,2,2,1,1,886.0,916.0,892.0,892.0,844.0,NO SUFICIENTE
2,1996.0,1,2,2,29,3,2,8.6,4,2,1,3,2,1084.0,1084.0,1108.0,1060.0,1084.0,NIVEL SATISFACTORIO
3,1995.0,1,2,2,29,3,2,9.8,5,5,2,2,1,1204.0,1276.0,1132.0,1252.0,1156.0,NIVEL SOBRESALIENTE
4,1996.0,1,2,2,29,3,1,7.3,4,3,2,2,2,1102.0,1108.0,1132.0,1036.0,1132.0,NIVEL SATISFACTORIO


In [7]:
df['ANO_NAC'].value_counts()

,count
ANO_NAC,
1996.0,413
1995.0,262
1994.0,74
1993.0,24
1992.0,10
1991.0,9
1990.0,6
1985.0,3
1987.0,3


In [8]:
import time
# Convertir la estructura de dato de ANO_NAC a una categoría
ano_actual = time.localtime().tm_year  # Se extrae el año actual

# Calcular la edad restando el año de nacimiento al año actual
df['EDAD'] = ano_actual - df['ANO_NAC']

# Crear categorías basadas en la edad: =<29 y >29
df['CATEGORIA_EDAD'] = pd.cut(df['EDAD'],
                              bins=[-float('inf'), 29, float('inf')],
                              labels=['1', '2'], # 1:=<29, >29
                              include_lowest=True)

In [9]:
df['CATEGORIA_EDAD'].value_counts()

,count
CATEGORIA_EDAD,
1,413
2,400


In [10]:
cols = ['CATEGORIA_EDAD'] + [col for col in df.columns if col != 'CATEGORIA_EDAD']
df = df[cols]

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 813 entries, 0 to 812
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   CATEGORIA_EDAD  813 non-null    category
 1   ANO_NAC         813 non-null    float64 
 2   SEXO            813 non-null    category
 3   IMP_EMO         813 non-null    category
 4   IMP_APR         813 non-null    category
 5   EDO_PROC        813 non-null    category
 6   REG_PROC        813 non-null    category
 7   MOD_BAC         813 non-null    category
 8   PROM_BAC        813 non-null    float64 
 9   DES_DURO        813 non-null    category
 10  ESCO_PAD        813 non-null    category
 11  SER_INTE        813 non-null    category
 12  BIEN_PC         813 non-null    category
 13  SER_AUTO        813 non-null    category
 14  ICNE            813 non-null    float64 
 15  IPMA            813 non-null    float64 
 16  IPAN            813 non-null    float64 
 17  IELE*           813 n

In [19]:
# Eliminamos las columnas modificadas
df.drop(columns=['ANO_NAC', 'EDAD', 'ICNE', 'IPMA', 'IPAN', 'IELE*', 'ICLE*'], inplace=True)

In [20]:
df.head()

,CATEGORIA_EDAD,SEXO,IMP_EMO,IMP_APR,EDO_PROC,REG_PROC,MOD_BAC,PROM_BAC,DES_DURO,ESCO_PAD,SER_INTE,BIEN_PC,SER_AUTO,class
0,1,1,2,2,29,3,1,7.0,3,6,2,1,2,2
1,1,2,1,2,29,3,1,9.3,5,2,2,1,1,2
2,1,1,2,2,29,3,2,8.6,4,2,1,3,2,0
3,2,1,2,2,29,3,2,9.8,5,5,2,2,1,1
4,1,1,2,2,29,3,1,7.3,4,3,2,2,2,0


### Transformación de datos

Categóricos -> Dummies

Numéricos -> Estandarizamos con Z-score

In [14]:
# Codificar la variable target
le = LabelEncoder()
df['class'] = le.fit_transform(df['class'])

In [15]:
df['class'].value_counts()

,count
class,
2,390
0,372
1,51


In [16]:
# Definir variables categóricas vs. numéricas
categorical_cols = ['CATEGORIA_EDAD', 'SEXO', 'IMP_EMO', 'IMP_APR', 'EDO_PROC', 'REG_PROC',
                    'MOD_BAC', 'DES_DURO', 'ESCO_PAD', 'SER_INTE', 'BIEN_PC', 'SER_AUTO']
numerical_cols = ['PROM_BAC']

In [17]:
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Preprocesamiento con ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

In [21]:
# Dividir los datos
# Dividir datos
X = df.drop('class', axis=1)
y = df['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Aplicar transformaciones
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

In [22]:
y_train.value_counts()

,count
class,
2,312
0,297
1,41


### Aplicar una técnica cuando hay desbalance

SMOTE (Synthetic Minority Oversampling Technique) es una técnica para balancear conjuntos de datos desbalanceados. Genera muestras sintéticas de la clase minoritaria interpolando entre ejemplos existentes, en lugar de replicarlos. Sirve para mejorar el rendimiento de modelos de machine learning al evitar el sesgo hacia la clase mayoritaria, aumentando la representación de la clase minoritaria sin sobreajuste.

In [23]:
from imblearn.over_sampling import SMOTE
# Verificar balance
print(y_train.value_counts())

# Aplicar SMOTE si hay desbalance
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_preprocessed, y_train)

class
2    312
0    297
1     41
Name: count, dtype: int64


In [24]:
y_train_balanced.value_counts()

,count
class,
0,312
2,312
1,312


### Modelos

In [25]:
rf = RandomForestClassifier(random_state=42)
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}

grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='f1_weighted')
grid_search.fit(X_train_balanced, y_train_balanced)
best_rf = grid_search.best_estimator_

In [26]:
svm = SVC(random_state=42)
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf']
}

grid_search_svm = GridSearchCV(svm, param_grid, cv=3, scoring='f1_weighted')
grid_search_svm.fit(X_train_balanced, y_train_balanced)
best_svm = grid_search_svm.best_estimator_

In [27]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_preprocessed.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(len(le.classes_), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=5)

history = model.fit(
    X_train_preprocessed, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop]
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - accuracy: 0.4056 - loss: 1.0719 - val_accuracy: 0.4462 - val_loss: 0.9010
Epoch 2/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.5322 - loss: 0.8796 - val_accuracy: 0.4769 - val_loss: 0.8821
Epoch 3/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5998 - loss: 0.8197 - val_accuracy: 0.5231 - val_loss: 0.8661
Epoch 4/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5715 - loss: 0.8267 - val_accuracy: 0.5769 - val_loss: 0.8454
Epoch 5/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5818 - loss: 0.8151 - val_accuracy: 0.5231 - val_loss: 0.8523
Epoch 6/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5814 - loss: 0.7971 - val_accuracy: 0.5538 - val_loss: 0.8342
Epoch 7/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6247 - loss: 0.7434 - val_accuracy: 0.5308 - val_loss: 0.8516
Epoch 8/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6085 - loss: 0.7652 - val_accuracy: 0.5538 - v

### Evaluar

In [28]:
# Función para evaluar
def evaluate_model(model, X_test, y_test, is_ann=False):
    if is_ann:
        y_pred = np.argmax(model.predict(X_test), axis=1)
    else:
        y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))
    print("Accuracy:", accuracy_score(y_test, y_pred))

# Evaluar Random Forest
evaluate_model(best_rf, X_test_preprocessed, y_test)

# Evaluar SVM
evaluate_model(best_svm, X_test_preprocessed, y_test)

# Evaluar ANN
evaluate_model(model, X_test_preprocessed, y_test, is_ann=True)

              precision    recall  f1-score   support

           0       0.48      0.48      0.48        75
           1       0.00      0.00      0.00        10
           2       0.53      0.59      0.56        78

    accuracy                           0.50       163
   macro avg       0.34      0.36      0.35       163
weighted avg       0.47      0.50      0.49       163

Accuracy: 0.5030674846625767
              precision    recall  f1-score   support

           0       0.46      0.49      0.48        75
           1       0.20      0.10      0.13        10
           2       0.53      0.53      0.53        78

    accuracy                           0.48       163
   macro avg       0.40      0.37      0.38       163
weighted avg       0.48      0.48      0.48       163

Accuracy: 0.48466257668711654
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
              precision    recall  f1-score   support

           0       0.51      0.55      0.53        75
           1       0.00      0.0

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
